# Premier League

Skrapa senaste resultaten från Premier League från sidan <https://www.premierleague.com/tables>.

## Funktioner för web scraping

In [ ]:
# Installera bibliotek.
!pip install BeautifulSoup4
!pip install lxml

In [1]:
import urllib.request
from lxml import html
from bs4 import BeautifulSoup 

# Funktion för stt skrapa HTML från webbsida.
def scrapewebpage(url):
	# Öppna URL och hämta HTML.
	web = urllib.request.urlopen(url)

	# Kontrollera att inga fel uppstod när URL öppnades.
	if (web.getcode() == 200):
		html = web.read()
		return(html)
	else:
		print("Error %s reading %s" % str(web.getcode()), url)

# Hjälpfunktion som skrapar webbsidor och gör en soppa av allt ihop.
def makesoup(url):
	html = scrapewebpage(url)
	return(BeautifulSoup(html, "lxml"))

## Hämta sidan

In [2]:
# Hämta Premier League-tabellen.
pl = makesoup("https://www.premierleague.com/tables")

In [3]:
# Hämta bara första fliken (sidan innehåller tre flikar).
tab = pl.find("div", "mainTableTab")

# Skapa tom lista att spara resultaten i.
pl_lista = []

# Hämta varje tabellrad som finns i fliken.
for tr in tab.find_all("tr", ""):
    team = tr.find("span", "long")        # Plocka ut namnet.
    if team != None:                      # Kontrollera att namnet inte är tomt.
        team = team.get_text()            # Ta bort HTML, behåll bara text.
        
        points = tr.find("td", "points")  # Leta upp <td class="points>.
        points = points.get_text()        # Ta ut texten från <td class="points> ... </td>
        points = int(points)              # Gör om text till tal.
        
        gd = tr.find_all("td")[9]         # Leta upp kolumn 10 (eftersom Python börjar räkna från 0!).
        gd = gd.get_text()                # Ta bort HTML, behåll bara text.
        gd = gd.strip()                   # Rensa bort mellanslag/nya rader.
        gd = int(gd)                      # Gör om text till tal.

        # Skapa ett dictionary-objekt. Dictionary är en variabeltyp
        # som innehåller så kallade nyckelpar (namn & värde).
        # De kan användas för att samla information som hör ihop.
        # Läs mer här:
        # https://www.pythonforbeginners.com/dictionary/how-to-use-dictionaries-in-python/
        dic = { "team": team,
                "points": points,
                "gd": gd
              }
        
        # Lägger till dictionary sist i listan.
        pl_lista.append(dic)

In [4]:
# Kolla innehållet i vår lista.
pl_lista

[{'gd': 18, 'points': 20, 'team': 'Manchester City'},
 {'gd': 13, 'points': 20, 'team': 'Chelsea'},
 {'gd': 12, 'points': 20, 'team': 'Liverpool'},
 {'gd': 9, 'points': 18, 'team': 'Arsenal'},
 {'gd': 8, 'points': 18, 'team': 'Tottenham Hotspur'},
 {'gd': 4, 'points': 16, 'team': 'Bournemouth'},
 {'gd': 3, 'points': 15, 'team': 'Wolverhampton Wanderers'},
 {'gd': -1, 'points': 13, 'team': 'Manchester United'},
 {'gd': -1, 'points': 13, 'team': 'Watford'},
 {'gd': 2, 'points': 12, 'team': 'Leicester City'},
 {'gd': 1, 'points': 12, 'team': 'Everton'},
 {'gd': -2, 'points': 8, 'team': 'Burnley'},
 {'gd': -4, 'points': 8, 'team': 'Brighton and Hove Albion'},
 {'gd': -4, 'points': 7, 'team': 'Crystal Palace'},
 {'gd': -5, 'points': 7, 'team': 'West Ham United'},
 {'gd': -8, 'points': 5, 'team': 'Southampton'},
 {'gd': -12, 'points': 5, 'team': 'Fulham'},
 {'gd': -13, 'points': 3, 'team': 'Huddersfield Town'},
 {'gd': -7, 'points': 2, 'team': 'Newcastle United'},
 {'gd': -13, 'points': 2, '

In [5]:
# Hur många lag innehåller listan vi skapade?
len(pl_lista)

20

## Spara listan till en fil

En CSV-fil är en komma-separerad lista som kan öppnas i Excel.

Fördelen med CSV är att det är ett öppet format som fungerar överallt.

Principen med CSV är enkel. Varje lag är på en egen rad, och varje värde separeras med komma-tecken.

De tre första raderna i CSV-filen kommer med andra ord se ut så här:

```
Manchester City,20
Chelsea,20
Liverpool,20
```

Och de sparas i en fil som vi döper till **premier-league.csv**.

In [6]:
# Öppna en fil för skrivning.
f = open("premier-league.csv", "w") # w = write, a = append, r = read

# Gå igenom listan, rad för rad.
for dic in pl_lista:
    # Skapa en variabel med teamnamn följt av poäng
    # genom att hämta värdet från dictionary-objektet.
    text = dic["team"] + "," + str(dic["points"])
    
    # Skriv varje team till filen.
    # \n betyder att det ska vara en ny rad.
    f.write(text + "\n")

# Vi får inte glömma att stänga filen.    
f.close()